In [195]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2,RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

In [176]:
df = pd.read_csv("train.csv")
new_cols = ['C' + str(i) for i in range(1, 14 + 1)] + ['X' + str(i) for i in range(1, 19 + 2)]
new_cols[-1] = 'Y'
df.columns = new_cols

features = df.loc[:, df.columns != 'Y']
numerical_features = df.select_dtypes(include=['float64']).columns
categorical_features = df.select_dtypes(include=['object']).columns
target = df['Y']

df

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,X11,X12,X13,X14,X15,X16,X17,X18,X19,Y
0,V1,V1,V1,V1,V1,V1,V2,V1,V3,V5,...,0.352,0.073,-0.092,1.098,0.034,1.160,0.401,0.037,0.216,1
1,V1,V1,V1,V1,V1,V1,V1,V1,V2,V2,...,-1.933,-0.536,-0.092,1.098,0.034,0.716,0.401,0.724,0.216,-1
2,V1,V1,V1,V1,V1,V1,V1,V1,V2,V2,...,-0.762,-0.536,-0.092,1.098,0.034,0.716,0.401,0.712,0.216,-1
3,V1,V1,V1,V1,V1,V1,V1,V1,V1,V1,...,-0.505,-0.536,-0.092,0.182,0.034,0.716,0.401,0.393,0.216,1
4,V1,V1,V1,V1,V1,V1,V1,V1,V1,V1,...,0.409,-0.536,-0.092,0.731,0.034,0.716,0.401,0.724,0.216,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,NaN,NaN,NaN,V3,V1,NaN,NaN,NaN,V1,V4,...,NaN,NaN,NaN,-2.568,0.034,NaN,0.401,1.890,0.216,1
395,NaN,NaN,NaN,V3,V2,NaN,NaN,V1,V3,V2,...,NaN,NaN,NaN,-4.401,-2.684,NaN,0.401,1.391,0.216,1
396,NaN,NaN,NaN,V3,V2,NaN,NaN,NaN,V1,V4,...,NaN,NaN,NaN,0.548,4.926,NaN,0.401,0.992,0.216,1
397,NaN,NaN,NaN,V3,V1,NaN,NaN,NaN,V1,V8,...,NaN,NaN,NaN,-1.651,3.567,NaN,0.401,0.992,0.216,1


In [177]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2)

In [178]:
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

#TODO Experiment with strategies
iterative_numerical_imputer = IterativeImputer(max_iter=10)
iterative_numerical_imputer.fit(X_train[numerical_features])
X_train.loc[:, numerical_features] = iterative_numerical_imputer.transform(X_train.loc[:, numerical_features])
X_test.loc[:, numerical_features] = iterative_numerical_imputer.transform(X_test.loc[:, numerical_features])

categorical_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
categorical_imputer.fit(X_train[categorical_features])
X_train.loc[:, categorical_features] = categorical_imputer.transform(X_train.loc[:, categorical_features])
X_test.loc[:, categorical_features] = categorical_imputer.transform(X_test.loc[:, categorical_features])

C:\Users\Gábor\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\impute\_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"
C:\Users\Gábor\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
C:\Users\Gábor\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [179]:
X_train_len = len(X_train)
X = pd.concat([X_train, X_test])
X = pd.get_dummies(X)
X_train = X[:X_train_len]
X_test = X[X_train_len:]

In [180]:
#X.to_csv("save\X.csv", index=False)
#X_train.to_csv("save\X_train.csv", index=False)
#X_test.to_csv("save\X_test.csv", index=False)

In [181]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier

classifier1 = RandomForestClassifier(n_estimators=1000)
classifier1.fit(X_train, y_train)

classifier2 = ExtraTreesClassifier(n_estimators=1000)
classifier2.fit(X_train, y_train)

ExtraTreesClassifier(n_estimators=1000)

In [182]:
y_pred1 = classifier1.predict(X_test)
cm1 = confusion_matrix(y_test, y_pred1)

print("RandomForestClassifier")
print(cm1)
print(classification_report(y_test, y_pred1))


y_pred2 = classifier2.predict(X_test)
cm2 = confusion_matrix(y_test, y_pred2)

print("ExtraTreesClassifier")
print(cm2)
print(classification_report(y_test, y_pred2))

RandomForestClassifier
[[51  2]
 [10 17]]
              precision    recall  f1-score   support

          -1       0.84      0.96      0.89        53
           1       0.89      0.63      0.74        27

    accuracy                           0.85        80
   macro avg       0.87      0.80      0.82        80
weighted avg       0.86      0.85      0.84        80

ExtraTreesClassifier
[[46  7]
 [ 7 20]]
              precision    recall  f1-score   support

          -1       0.87      0.87      0.87        53
           1       0.74      0.74      0.74        27

    accuracy                           0.82        80
   macro avg       0.80      0.80      0.80        80
weighted avg       0.82      0.82      0.82        80



In [183]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred1))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred1))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred1)))

Mean Absolute Error: 0.3
Mean Squared Error: 0.6
Root Mean Squared Error: 0.7745966692414834


In [187]:
def evaluate_model(X_train, y_train, X_test, y_test, model):
	cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
	#model.fit(X_train, y_train)
	return cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

models, names = list(), list()
# SVM
models.append(SVC(gamma='auto'))
names.append('SVM')
# KNN
models.append(KNeighborsClassifier())
names.append('KNN')
# Neutral Network
#models.append(MLPClassifier(solver = 'adam', max_iter=1000, tol=0.000001, early_stopping=True, validation_fraction=0.1, n_iter_no_change=20))
#names.append('NEU')
# Bagging
models.append(BaggingClassifier(n_estimators=1000))
names.append('BAG')
# RF
models.append(RandomForestClassifier(n_estimators=1000))
names.append(' RF')
# ET
models.append(ExtraTreesClassifier(n_estimators=1000))
names.append(' ET')


results = list()
for i in range(len(models)):
	scores = evaluate_model(X_train, y_train, X_test, y_test, models[i])
	results.append(scores)
	print('>%s %.3f (%.3f)' % (names[i], scores.mean(), scores.std()))
	

>SVM 0.745 (0.044)
>KNN 0.758 (0.052)
>BAG 0.807 (0.019)
> RF 0.820 (0.036)
> ET 0.825 (0.053)


In [193]:
# TODO Test GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import pprint

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [198]:
import time

rf = RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation, search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
start = time.time()

rf_random.fit(X_train, y_train)

end = time.time()
print("Elapsed time: ", end - start, " seconds, which is: ", (end - start) / 60, " minutes")

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Elapsed time:  108.89499998092651  seconds, which is:  1.8149166663487752  minutes


In [199]:
rf_random.best_params_

{'n_estimators': 1800,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 20,
 'bootstrap': False}

In [206]:
def evaluate(model, X_test, y_test):
    y_pred = model.predict(X_test)
    errors = abs(y_pred - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    print('\nModel Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    print(cm1)
    print(classification_report(y_test, y_pred))
    
    return accuracy
    
base_model = RandomForestClassifier(n_estimators = 1000)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))


Model Performance
Average Error: 0.3250 degrees.
Accuracy = 82.50%.
[[51  2]
 [10 17]]
              precision    recall  f1-score   support

          -1       0.83      0.94      0.88        53
           1       0.85      0.63      0.72        27

    accuracy                           0.84        80
   macro avg       0.84      0.79      0.80        80
weighted avg       0.84      0.84      0.83        80


Model Performance
Average Error: 0.3250 degrees.
Accuracy = 87.50%.
[[51  2]
 [10 17]]
              precision    recall  f1-score   support

          -1       0.84      0.92      0.88        53
           1       0.82      0.67      0.73        27

    accuracy                           0.84        80
   macro avg       0.83      0.80      0.81        80
weighted avg       0.84      0.84      0.83        80

Improvement of 6.06%.


In [185]:
"""
clf = RandomForestClassifier()
clf.fit(X, target)

plt.figure(figsize=(20,8))
plt.bar(X.columns, clf.feature_importances_)
plt.xticks(rotation=45)
"""

'\nclf = RandomForestClassifier()\nclf.fit(X, target)\n\nplt.figure(figsize=(20,8))\nplt.bar(X.columns, clf.feature_importances_)\nplt.xticks(rotation=45)\n'

In [186]:
#GridSearchCV?
# TODO https://datascience.stackexchange.com/questions/75345/need-help-understanding-data-leakage
# TODO https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74